# **Import Libraries**

In [1]:
import warnings

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import transforms
from torchvision.datasets import MNIST

from torch.utils.data import DataLoader

from tqdm.auto import tqdm

# **Ignoring Any Warning Messages**

In [2]:
warnings.filterwarnings('ignore')

# **Define Important Hyperparameters**

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
lr = 3e-4
z_dim = 64
img_dim = 28 * 28
num_epochs = 80
fixed_noise = torch.rand(batch_size, z_dim).to(device)

# **Create Discriminator**

In [4]:
class Discriminator(nn.Module):
  def __init__(self, in_features):
    super().__init__()

    self.net = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.LeakyReLU(.01),

        nn.Linear(512, 256),
        nn.LeakyReLU(.01),

        nn.Linear(256, 128),
        nn.LeakyReLU(.01),

        nn.Linear(128, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.net(x)

# **Create Generator**

In [5]:
class Generator(nn.Module):
  def __init__(self, z_dim, img_dim):
    super().__init__()

    self.net = nn.Sequential(
        nn.Linear(z_dim, 128),
        nn.LeakyReLU(.01),

        nn.Linear(128, 256),
        nn.LeakyReLU(.01),

        nn.Linear(256, 512),
        nn.LeakyReLU(.01),

        nn.Linear(512, img_dim),
        nn.Tanh(),
    )

  def forward(self, x):
    return self.net(x)

# **Generate Dataloader**

### **Define Transformation**

In [6]:
transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((.5, ), (.5, ))
    ]
)

### **Create Dataloader**

In [7]:
dataloader = DataLoader(MNIST('.', download = True, transform = transforms), batch_size = batch_size, shuffle = True)

# **Training Loop**

In [8]:
disc = Discriminator(img_dim).to(device)
gen  = Generator(z_dim, img_dim).to(device)

opt_disc = optim.Adam(disc.parameters(), lr = lr)
opt_gen  = optim.Adam(gen.parameters(),  lr = lr)

criterion = nn.BCELoss()

for epoch in range(num_epochs):
  for real, _ in tqdm(dataloader):

    batch_size = real.shape[0]
    real = real.view(batch_size, -1).to(device)

    noise = torch.randn(batch_size, z_dim).to(device)
    fake  = gen(noise)

    disc_real  = disc(real).view(-1)
    disc_fake = disc(fake).view(-1)

    lossD_real  = criterion(disc_real,  torch.ones_like(disc_real))
    lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake) / 2

    disc.zero_grad()
    lossD.backward(retain_graph=True)
    opt_disc.step()

    output = disc(fake).view(-1)

    lossG = criterion(output, torch.ones_like(output))

    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

  print(f"epoch{epoch + 1}:  lossD:{lossD : .4f}   lossG:{lossG : .4f}")

  with torch.no_grad():
    fake = gen(fixed_noise).reshape(-1, 1, 28, 28).to(device)
    real = next(iter(dataloader))[0].reshape(-1, 1, 28, 28).to(device)

    img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
    img_grid_real = torchvision.utils.make_grid(real, normalize=True)

    torchvision.utils.save_image(img_grid_fake, "fake_grid.png")
    torchvision.utils.save_image(img_grid_real, "real_grid.png")

  0%|          | 0/1875 [00:00<?, ?it/s]

epoch1:  lossD: 0.4002   lossG: 1.7359


  0%|          | 0/1875 [00:00<?, ?it/s]

epoch2:  lossD: 0.0880   lossG: 4.9819


  0%|          | 0/1875 [00:00<?, ?it/s]

...

epoch78:  lossD: 0.3636   lossG: 1.2525


  0%|          | 0/1875 [00:00<?, ?it/s]

epoch79:  lossD: 0.4977   lossG: 1.5100


  0%|          | 0/1875 [00:00<?, ?it/s]

epoch80:  lossD: 0.4464   lossG: 1.7153
